# Privilege Escalation - RogueWinRM Exploit

#### Author: Rifqi Ardia Ramadhan

This is a known Windows Exploit in the very recent Windows version (1909). Read more about the technical detail [here](https://decoder.cloud/2019/12/06/we-thought-they-were-potatoes-but-they-were-beans/). In short it's exploiting a Windows service (BITS) that's just lying around that happen to have a privilege, and impersonating the credential it gave us.

## Use Case
### Objective
Identify the RogueWinRM Exploit using the characteristics of invoking BITS and then running service that uses port 5985, at a significantly near timestamp. WinRM that is a big part of this exploit is a Windows Remote Management service that by default is disabled by system. Although this require `Service Account` to access first, this is still a possible privilege escalation exploit.

### Logs Requirements
Microsoft Windows Sysmon

### Characteristics and Behaviour
| No | Characteristic | Note | Why|
|----|----------------|------|---|
|  1 | A RogueWinRM.exe process logged               | Optional | Actor might rename the file to another name|
|  2 | BITS request invoked by svchost.exe | Non-optional     | |
|  3 | A TCP connection is open with destination port 5985        | Sequential     | After step 2|
| 4 | Running whoami command as a part of usual privilege checking | Optional | Actor might not use this command |

### References
https://attack.mitre.org/techniques/T1134/001

https://attack.mitre.org/techniques/T1197

### Sample
Dataset acquired from [EVTX-ATTACK-SAMPLES](https://github.com/sbousseaden/EVTX-ATTACK-SAMPLES)

In [75]:
import Evtx.Evtx as evtx
import Evtx.Views as e_views
import xml.dom.minidom as minidom
from xml.etree.ElementTree import XML
import re
        
def get_events(input_file, parse_xml=False):
    """Opens a Windows Event Log and returns XML information from
    the event record.

    Arguments:
        input_file (str): Path to evtx file to open
        parse_xml (bool): If True, return an lxml object, otherwise a string

    Yields:
        (generator): XML information in object or string format

    Examples:
        >>> for event_xml in enumerate(get_events("System.evtx")):
        >>>     print(event_xml)

    """
    with evtx.Evtx(input_file) as event_log:
        for record in event_log.records():
            if parse_xml:
                yield record.lxml()
            else:
                yield record.xml()

# all of the above is just to parse the EVTX file as XML
                
filename = "logs/RogueWinRM.evtx"
xmloutput = ""
for event_xml in enumerate(get_events(filename)):
    if "BITS" in event_xml[1]:                               #Check if BITS was invoked
        parsedxml = minidom.parseString(event_xml[1])
        prettyxml = parsedxml.toprettyxml()
        print(event_xml[0], prettyxml)


-------------------------------------------------------------------------------------
1 <?xml version="1.0" ?>
<Event xmlns="http://schemas.microsoft.com/win/2004/08/events/event">
	<System>
		<Provider Guid="{5770385f-c22a-43e0-bf4c-06f5698ffbd9}" Name="Microsoft-Windows-Sysmon"/>
		

		<EventID Qualifiers="">1</EventID>
		

		<Version>5</Version>
		

		<Level>4</Level>
		

		<Task>1</Task>
		

		<Opcode>0</Opcode>
		

		<Keywords>0x8000000000000000</Keywords>
		

		<TimeCreated SystemTime="2020-05-24 01:13:48.864511"/>
		

		<EventRecordID>196366</EventRecordID>
		

		<Correlation ActivityID="" RelatedActivityID=""/>
		

		<Execution ProcessID="2812" ThreadID="3656"/>
		

		<Channel>Microsoft-Windows-Sysmon/Operational</Channel>
		

		<Computer>MSEDGEWIN10</Computer>
		

		<Security UserID="S-1-5-18"/>
		

	</System>
	

	<EventData>
		<Data Name="RuleName"/>
		

		<Data Name="UtcTime">2020-05-24 01:13:48.854</Data>
		

		<Data Name="ProcessGuid">{747f3d96-ca4c-5ec9-0000-001093d53700}

In [76]:
print("-------------------------------------------------------------------------------------")
for event_xml in enumerate(get_events(filename)):
    if "NT AUTHORITY\\SYSTEM" in event_xml[1] and '"DestinationPort">5985' in event_xml[1]:   #Check if there is SYSTEM running in Port 5985
        parsedxml = minidom.parseString(event_xml[1])
        prettyxml = parsedxml.toprettyxml()
        print(event_xml[0], prettyxml)

-------------------------------------------------------------------------------------
4 <?xml version="1.0" ?>
<Event xmlns="http://schemas.microsoft.com/win/2004/08/events/event">
	<System>
		<Provider Guid="{5770385f-c22a-43e0-bf4c-06f5698ffbd9}" Name="Microsoft-Windows-Sysmon"/>
		

		<EventID Qualifiers="">3</EventID>
		

		<Version>5</Version>
		

		<Level>4</Level>
		

		<Task>3</Task>
		

		<Opcode>0</Opcode>
		

		<Keywords>0x8000000000000000</Keywords>
		

		<TimeCreated SystemTime="2020-05-24 01:13:51.206106"/>
		

		<EventRecordID>196373</EventRecordID>
		

		<Correlation ActivityID="" RelatedActivityID=""/>
		

		<Execution ProcessID="2812" ThreadID="3488"/>
		

		<Channel>Microsoft-Windows-Sysmon/Operational</Channel>
		

		<Computer>MSEDGEWIN10</Computer>
		

		<Security UserID="S-1-5-18"/>
		

	</System>
	

	<EventData>
		<Data Name="RuleName"/>
		

		<Data Name="UtcTime">2020-05-24 01:13:50.129</Data>
		

		<Data Name="ProcessGuid">{747f3d96-ca4c-5ec9-0000-001093d53700}